# Full example with the Hugging Face Transformers package

This notebook shows how to train a model (GPT2) and generate music from it, using the Hugging Face Transformers package.

## Setup Environment

In [6]:
#@title Install all dependencies (run only once per session)

!nvidia-smi

!pip -q install miditok miditoolkit torch torchtoolkit transformers accelerate evaluate tqdm scikit-learn tensorboard

![ -d "maestro-v3.0.0" ] || { wget "https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip" -O "maestro-v3.0.0.zip" && unzip -q "maestro-v3.0.0.zip" && rm 'maestro-v3.0.0.zip' && ln -s "maestro-v3.0.0" "Maestro"; }


Wed Nov  8 22:42:22 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.02              Driver Version: 545.29.02    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     Off | 00000000:01:00.0 Off |                  N/A |
| 34%   52C    P2              61W / 250W |   6600MiB / 11264MiB |      3%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable 

In [ ]:
from pathlib import Path
from copy import deepcopy

from torch import Tensor, argmax
from torch.utils.data import DataLoader
from torch.cuda import is_available as cuda_available, is_bf16_supported
from torch.backends.mps import is_available as mps_available
from torchtoolkit.data import create_subsets
from transformers import GPT2LMHeadModel, GPT2Config, Trainer, TrainingArguments, GenerationConfig
from evaluate import load as load_metric
from miditok import REMI, TokenizerConfig
from miditok.pytorch_data import DatasetTok, DataCollator
from tqdm import tqdm

## Convert MIDI files to tokens, and load them for training

In [2]:
# Our tokenizer's configuration
PITCH_RANGE = (21, 109)
BEAT_RES = {(0, 1): 8, (1, 2): 4, (2, 4): 2, (4, 8): 1}
NB_VELOCITIES = 24
SPECIAL_TOKENS = ["PAD", "MASK", "BOS", "EOS"]
USE_CHORDS = False
USE_RESTS = False
USE_TEMPOS = True
USE_TIME_SIGNATURE = False
USE_PROGRAMS = False
NB_TEMPOS = 32
TEMPO_RANGE = (50, 200)  # (min_tempo, max_tempo)
TOKENIZER_PARAMS = {
    "pitch_range": PITCH_RANGE,
    "beat_res": BEAT_RES,
    "nb_velocities": NB_VELOCITIES,
    "special_tokens": SPECIAL_TOKENS,
    "use_chords": USE_CHORDS,
    "use_rests": USE_RESTS,
    "use_tempos": USE_TEMPOS,
    "use_time_signatures": USE_TIME_SIGNATURE,
    "use_programs": USE_PROGRAMS,
    "nb_tempos": NB_TEMPOS,
    "tempo_range": TEMPO_RANGE,
}
config = TokenizerConfig(**TOKENIZER_PARAMS)


# Creates the tokenizer convert MIDIs to tokens
tokenizer = REMI(config)  # REMI
 
tokens_path = Path('Maestro_tokens_no_bpe')
if not Path(tokens_path).exists():      # don't re-do this unless necessary, as it takes a while
    midi_paths = list(Path('Maestro').glob('**/*.mid')) + list(Path('Maestro').glob('**/*.midi'))
    tokenizer.tokenize_midi_dataset(midi_paths, tokens_path)  

# Learn and apply BPE to data we just tokenized
tokens_bpe_path = Path('Maestro_tokens_bpe')
tokens_bpe_path.mkdir(exist_ok=True, parents=True)
tokenizer.learn_bpe(
    vocab_size=10000,
    tokens_paths=list(tokens_path.glob("**/*.json")),
    start_from_empty_voc=False,
)
tokenizer.save_params("tokenizer_bpe.conf")
tokenizer.apply_bpe_to_dataset(
    tokens_path,
    tokens_bpe_path,
)

# Loads tokens and create data loaders for training
tokens_paths = list(tokens_bpe_path.glob("**/*.json"))
dataset = DatasetTok(
    tokens_paths, max_seq_len=512, min_seq_len=384, one_token_stream=False,
)
subset_train, subset_valid = create_subsets(dataset, [0.3])

Loading token files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1276/1276 [00:02<00:00, 468.38it/s]


Loading data: Maestro_tokens_bpe/2006: 100%|███████████████████████████████████████████████████████████████████████████████████| 1276/1276 [00:01<00:00, 821.79it/s]


## Create the model

We will use the [GPT2 implementation of Hugging Face](https://huggingface.co/docs/transformers/model_doc/gpt2). This 
Feel free to explore the documentation and source code to dig deeper.

In [3]:
# Creates model
config = GPT2Config(
    vocab_size=len(tokenizer),
    n_positions=2048,
    n_embd=512,
    n_layer=8,
    n_head=8,
    n_inner=2048,
    resid_pdrop=.1,
    embd_pdrop=.1,
    attn_pdrop=.1,
    padding_token_id=tokenizer['PAD_None'],
    bos_token_id=tokenizer['BOS_None'],
    eos_token_id=tokenizer['EOS_None'],
)
model = GPT2LMHeadModel(config)

## Train it

In [4]:
metrics = {metric: load_metric(metric) for metric in ["accuracy"]}

def compute_metrics(eval_pred):
    """Computes metrics for pretraining.
    Must use proprocess_logits function that converts logits to predictions (argmax or sampling).

    :param eval_pred: EvalPrediction containing predictions and labels
    :return: metrics
    """
    predictions, labels = eval_pred
    not_pad_mask = labels != -100
    labels, predictions = labels[not_pad_mask], predictions[not_pad_mask]
    return metrics["accuracy"].compute(predictions=predictions.flatten(), references=labels.flatten())

def preprocess_logits(logits: Tensor, _: Tensor) -> Tensor:
    """Preprocesses the logits before accumulating them during evaluation.
    This allows to significantly reduce the memory usage and make the training tractable.
    """
    pred_ids = argmax(logits, dim=-1)  # long dtype
    return pred_ids

# Create config for the Trainer
USE_CUDA = cuda_available()
if not cuda_available():
    FP16 = FP16_EVAL = BF16 = BF16_EVAL = False
elif is_bf16_supported():
    BF16 = BF16_EVAL = True
    FP16 = FP16_EVAL = False
else:
    BF16 = BF16_EVAL = False
    FP16 = FP16_EVAL = True
USE_MPS = not USE_CUDA and mps_available()
training_config = TrainingArguments(
    "runs", False, True, True, False, "steps",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=48,
    gradient_accumulation_steps=3,
    eval_accumulation_steps=None,
    eval_steps=1000,
    learning_rate=1e-4,
    weight_decay=0.01,
    max_grad_norm=3.0,
    max_steps=100000,
    lr_scheduler_type="cosine_with_restarts",
    warmup_ratio=0.3,
    log_level="debug",
    logging_strategy="steps",
    logging_steps=20,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=5,
    no_cuda=not USE_CUDA,
    seed=444,
    fp16=FP16,
    fp16_full_eval=FP16_EVAL,
    bf16=BF16,
    bf16_full_eval=BF16_EVAL,
    load_best_model_at_end=True,
    label_smoothing_factor=0.,
    optim="adamw_torch",
    report_to=["tensorboard"],
    gradient_checkpointing=True,
)

collator = DataCollator(tokenizer["PAD_None"], tokenizer["BOS_None"], tokenizer["EOS_None"], copy_inputs_as_labels=True)
trainer = Trainer(
    model=model,
    args=training_config,
    data_collator=collator,
    train_dataset=subset_train,
    eval_dataset=subset_valid,
    compute_metrics=compute_metrics,
    callbacks=None,
    preprocess_logits_for_metrics=preprocess_logits,
)

# Training
train_result = trainer.train()
trainer.save_model()  # Saves the tokenizer too
trainer.log_metrics("train", train_result.metrics)
trainer.save_metrics("train", train_result.metrics)
trainer.save_state()

max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend
Currently training with a batch size of: 16
***** Running training *****
  Num examples = 14,765
  Num Epochs = 326
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 48
  Gradient Accumulation steps = 3
  Total optimization steps = 100,000
  Number of trainable parameters = 31,388,672
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/shawley/envs/dlaie/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  

Step,Training Loss,Validation Loss,Accuracy
1000,8.237400,8.223081,0.049883
2000,7.649700,7.635656,0.040801
3000,6.929200,6.872255,0.000421
4000,6.136600,6.079532,0.000118
5000,5.744400,5.688006,0.000207
6000,5.471800,5.418881,0.000154
7000,5.274900,5.195198,0.000123
8000,5.099300,5.015362,0.000151
9000,4.934500,4.856715,0.000202
10000,4.798900,4.707597,0.000363


***** Running Evaluation *****
  Num examples = 6327
  Batch size = 48
Saving model checkpoint to runs/checkpoint-1000
Configuration saved in runs/checkpoint-1000/config.json
Configuration saved in runs/checkpoint-1000/generation_config.json
Model weights saved in runs/checkpoint-1000/pytorch_model.bin
/home/shawley/envs/dlaie/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
***** Running Evaluation *****
  Num examples = 6327
  Batch size = 48
Saving model checkpoint to runs/checkpoint-2000
Configuration saved in runs/checkpoint-2000/config.json
Configuration saved in runs/checkpoint-2000/generati

***** train metrics *****
  epoch                    =      325.03
  total_flos               = 347627597GF
  train_loss               =      2.8814
  train_runtime            = 21:22:36.00
  train_samples_per_second =      62.373
  train_steps_per_second   =       1.299


## Generate music

In [15]:
(gen_results_path := Path('gen_res')).mkdir(parents=True, exist_ok=True)
generation_config = GenerationConfig(
    max_new_tokens=768, #512,  # extends samples by 512 tokens
    num_beams=1,        # no beam search
    do_sample=True,     # but sample instead
    temperature=0.9,
    top_k=15,
    top_p=0.95,
    epsilon_cutoff=3e-4,
    eta_cutoff=1e-3,
    pad_token_id=config.padding_token_id,
)

# Here the sequences are padded to the left, so that the last token along the time dimension
# is always the last token of each seq, allowing to efficiently generate by batch
collator.pad_on_left = True
collator.eos_token = None
dataloader_test = DataLoader(subset_valid, batch_size=16, collate_fn=collator)
model.eval()
count = 0
for batch in tqdm(dataloader_test, desc='Testing model / Generating results'):  # (N,T)
    res = model.generate(
        inputs=batch["input_ids"].to(model.device),
        attention_mask=batch["attention_mask"].to(model.device),
        generation_config=generation_config)  # (N,T)

    # Saves the generated music, as MIDI files and tokens (json)
    for prompt, continuation in zip(batch["input_ids"], res):
        generated = continuation[len(prompt):]
        midi = tokenizer.tokens_to_midi([deepcopy(generated.tolist())], time_division=384)
        tokens = [generated, prompt, continuation]  # list compr. as seqs of dif. lengths
        tokens = [seq.tolist() for seq in tokens]
        for tok_seq in tokens[1:]:
            _midi = tokenizer.tokens_to_midi([deepcopy(tok_seq)], time_division=384)
            midi.instruments.append(_midi.instruments[0])
        midi.instruments[0].name = f'Continuation of original sample ({len(generated)} tokens)'
        midi.instruments[1].name = f'Original sample ({len(prompt)} tokens)'
        midi.instruments[2].name = f'Original sample and continuation'
        midi.dump(gen_results_path / f'{count}.mid')
        tokenizer.save_tokens(tokens, gen_results_path / f'{count}.json') 

        count += 1

Testing model / Generating results: 100%|██████████| 396/396 [03:36<00:00,  1.83it/s]
